This code will scrape the web data in order to create a local dataset (.csv) which will be used throughout the project. Note that this code will store some parts of the data locally to your computer. These parts will contain info that - while technically being public - would rather not want to be forwarded. That's why after seeing the dashboard results, I urge any reader to delete any resulting data.

In order for the code to work, you must have a folder to which the data will go to be stored. Please create an empty folder called "json_data" in the directory of this notebook.

### Imports:

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
from datetime import datetime
from ast import literal_eval
import numpy as np
#The following only stores a list of all heroes and their misspells. We will import it for conciseness of this notebook
from hero_list import *

def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s
    
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# LH Dataset collection:

In [2]:
#Making the API connection:
players_url = 'https://liga-heroes.pl/api/players'
dicto_players = requests.get(players_url).json()

In [3]:
#Making the API conenction:
starting_url = 'https://liga-heroes.pl/api/matches'
dicto = requests.get(starting_url).json()

In [4]:
#Changing the dictionary obtained from the API into a pandas dataframe:
df_matches = pd.DataFrame(dicto)
df_matches = df_matches[df_matches['winner_id'] != 0]
df_matches = df_matches[['host_id', 'guest_id', 'winner_id', 'details', 'updated_at']]
df_matches

,host_id,guest_id,winner_id,details,updated_at
0,206,224,224,"[{""giveup"": ""206"", ""template"": ""h3dm1"", ""hero_...",2021-04-29T16:54:05.000000Z
1,208,222,222,"[{""giveup"": ""208"", ""template"": ""h3dm1"", ""hero_...",2021-04-29T16:54:05.000000Z
2,210,220,220,"[{""giveup"": ""210"", ""template"": ""6lm10a"", ""hero...",2021-04-29T16:54:05.000000Z
3,212,218,212,"[{""giveup"": ""218"", ""template"": ""mt_fw"", ""hero_...",2021-04-29T16:54:05.000000Z
4,214,216,216,"[{""giveup"": ""214"", ""template"": ""nostalgia"", ""h...",2021-04-29T16:54:05.000000Z
...,...,...,...,...,...
5621,343,752,752,"[{""overtime"": false, ""reporter"": 462, ""templat...",2023-03-26T21:03:03.000000Z
5622,757,791,757,"[{""fail"": true, ""giveup"": true, ""fb_turn"": ""12...",2023-03-20T16:08:52.000000Z
5623,736,396,736,"[{""fail"": false, ""giveup"": true, ""fb_turn"": ""2...",2023-03-26T10:43:16.000000Z
5627,770,762,770,"[{""overtime"": false, ""reporter"": 487, ""templat...",2023-03-22T18:45:54.000000Z


Here, notice an important feature: the player of ID=1 is called 'Remis'. 'Remis' is the Polish term for 'draw/tie' in games. In case of a draw, the fake "remis" player will be awarded the win

In [5]:
#Changing the dictionary obtained from the API into a pandas dataframe:
df_players = pd.DataFrame(dicto_players)
df_players = df_players.drop([0], axis=0)
df_players = df_players.set_index("id")
df_players

,name
id,
1,Remis
206,Gumis
207,Tsar Konrad
208,Aproh
209,Majestatyczny
...,...
789,amator
790,Kenye
791,mamaragan


In [6]:
def replace_ids_with_names(number: int, df_players: pd.DataFrame):
    '''Converts IDs to names for successful merging with h3gg dataset'''
    return df_players['name'][number]

In [7]:
df_matches['host_id'] = df_matches['host_id'].apply(replace_ids_with_names, args=(df_players, ))
df_matches['guest_id'] = df_matches['guest_id'].apply(replace_ids_with_names, args=(df_players, ))
df_matches['winner_id'] = df_matches['winner_id'].apply(replace_ids_with_names, args=(df_players, ))
df_matches = df_matches.reset_index(drop=True)

In [8]:
df_matches

,host_id,guest_id,winner_id,details,updated_at
0,Gumis,Dawidu_91,Dawidu_91,"[{""giveup"": ""206"", ""template"": ""h3dm1"", ""hero_...",2021-04-29T16:54:05.000000Z
1,Aproh,RitoSux,RitoSux,"[{""giveup"": ""208"", ""template"": ""h3dm1"", ""hero_...",2021-04-29T16:54:05.000000Z
2,Wraku,Kexibq2134,Kexibq2134,"[{""giveup"": ""210"", ""template"": ""6lm10a"", ""hero...",2021-04-29T16:54:05.000000Z
3,Ziemian,Asis321,Ziemian,"[{""giveup"": ""218"", ""template"": ""mt_fw"", ""hero_...",2021-04-29T16:54:05.000000Z
4,Kadziel,Hektor_96,Hektor_96,"[{""giveup"": ""214"", ""template"": ""nostalgia"", ""h...",2021-04-29T16:54:05.000000Z
...,...,...,...,...,...
4951,Blackyyo,Zator,Zator,"[{""overtime"": false, ""reporter"": 462, ""templat...",2023-03-26T21:03:03.000000Z
4952,Ercho,mamaragan,Ercho,"[{""fail"": true, ""giveup"": true, ""fb_turn"": ""12...",2023-03-20T16:08:52.000000Z
4953,balkan13,bzzzz,balkan13,"[{""fail"": false, ""giveup"": true, ""fb_turn"": ""2...",2023-03-26T10:43:16.000000Z
4954,Kosixon,mixau32,Kosixon,"[{""overtime"": false, ""reporter"": 487, ""templat...",2023-03-22T18:45:54.000000Z


In [9]:
def str_to_datetime(text: str):
    '''Remember to re-parse the csv with dates as datetime when loading later on
    The -8 helps combat the .0000000Z ending
    Works with LH, not h3gg'''
    
    return datetime.strptime(text[:-8], '%Y-%m-%dT%H:%M:%S')


def str_to_dict(text: str):
    '''I made it work with BO3 matches too - both the ones that actually took 3 games and the ones that took 2 games'''
    try:
        return [json.loads(text[1:-1])]
    except:
        part_1 = json.loads(text[1:text.find('}')+1])
        text = text[text.find('}')+3:]
        part_2 = json.loads(text[:text.find('}')+1])
        text = text[text.find('}')+3:]
        if len(text) > 1:
            part_3 = json.loads(text[:-1])
            return [part_1, part_2, part_3]
        else:
            return [part_1, part_2]

        
df_matches['updated_at'] = df_matches['updated_at'].apply(str_to_datetime)
df_matches['details'] = df_matches['details'].apply(str_to_dict)

In [10]:
def extract_bo3_matches(df: pd.DataFrame):
    '''splits each row containing 3 games into 3 rows containing 1 game each'''
    
    list_of_games_to_append = []
    
    #Iterating over the rows (I know it is inefficient):
    for i in range(df.shape[0]):
        row = list(df.loc[i])
    
        #If spotted a BO3 match
        if len(row[3]) > 1:
            
            #Copy the match, but only use the second setup. Paste it into the new games list
            row2 = row.copy()
            
            row2[3] = [row[3][1]]
            list_of_games_to_append.append(row2)
            
            #If the BO3 match ended in a 2-1 or 1-2:
            if len(row[3]) > 2:
                row2 = row.copy()
                row2[3] = [row[3][2]]
                list_of_games_to_append.append(row2)
                
            #Only leave the first setup here in the original match:
            df.loc[i][3] = [df.loc[i][3][0]]
            
    return list_of_games_to_append

In [11]:
additional_games = extract_bo3_matches(df_matches)

df = pd.DataFrame({'host_id': [0]*len(additional_games), 'guest_id': [0]*len(additional_games),
                  'winner_id': [0]*len(additional_games), 'details': [0]*len(additional_games),
                   'updated_at': [0]*len(additional_games)})

for i in range(len(additional_games)):
    df.loc[i] = additional_games[i]
    
df_lh = pd.concat([df_matches, df], axis=0).reset_index(drop=True)

<ipython-input-10-f87c8d447fad>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i][3] = [df.loc[i][3][0]]


In [12]:
def deleting_useless_info(array: list):
    '''Strips the dictionary in df[details] off useless information'''
    
    dic = array[0]
    for header in ['fail', 'giveup', 'reporter', 'winner_id', 'overtime']:
        if header in dic.keys():
            del dic[header]

    return dic

In [13]:
df_lh = df_lh.sort_values(by=["updated_at"]).reset_index(drop=True)
df_lh['details'] = df_lh['details'].apply(deleting_useless_info)
df_lh

,host_id,guest_id,winner_id,details,updated_at
0,Gumis,Dawidu_91,Dawidu_91,"{'template': 'h3dm1', 'hero_host': 'Adelaide',...",2021-04-29 16:54:05
1,michail117,[SDK] Ravciozo,[SDK] Ravciozo,"{'template': 'nostalgia', 'hero_host': 'Solmyr...",2021-04-29 16:54:05
2,Woźny888,Lexiav,Woźny888,"{'template': 'h3dm1', 'hero_host': 'Gelare', '...",2021-04-29 16:54:05
3,Lexiav,michail117,michail117,"{'template': '6lm10a', 'hero_host': 'Valeska',...",2021-04-29 16:54:05
4,[SDK] Ravciozo,Kamil315,[SDK] Ravciozo,"{'template': '--Szablon--', 'walkower': '228',...",2021-04-29 16:54:05
...,...,...,...,...,...
5245,OstelikS,konsamaxi,konsamaxi,"{'template': 'w/o', 'host_gold': 0, 'guest_gol...",2023-04-01 19:06:04
5246,Roberto,OstelikS,Roberto,"{'fb_turn': '133', 'bid_host': 1800, 'template...",2023-04-01 19:06:43
5247,mitsurugi321,sudokuwbetlejem,sudokuwbetlejem,"{'fb_turn': '133', 'bid_host': 2000, 'template...",2023-04-01 21:20:29
5248,ProtosArchon,Narmalcil,Narmalcil,"{'fb_turn': '124', 'template': 'h3dm3', 'hero_...",2023-04-01 21:31:52


In [14]:
#changing the order of columns + sometimes the names
df_lh = pd.DataFrame({'player_one': df_lh['host_id'], 'score_player_one': [0]*df_lh.shape[0],
                    'player_two': df_lh['guest_id'], 'score_player_two': [0]*df_lh.shape[0],
                      'winner': df_lh['winner_id'],'details': df_lh['details'], 'updated_at': df_lh['updated_at']})

#This is one big error in the database, it's a game from 2020 with a glitch in the winner tab, I'm fixing it:
df_lh.loc[1607, 'winner'] = 'whitepill'
#And this is another error, but it's a game where I played, so I looked up the data to fill it in too:
df_lh.loc[1]['details']['fb_turn'] = '134'

In [15]:
#Converting the scores from LH-style to H3GG-style

for i in range(df_lh.shape[0]):
    row = df_lh.loc[i]

    if row['winner'] == row['player_one']:
        row['score_player_one'] = 1
    elif row['winner'] == row['player_two']:
        row['score_player_two'] = 1
    elif row['winner'] == 'Remis':
        row['score_player_two'] = 0.5
        row['score_player_one'] = 0.5
    else:
        print("Undefined winner in row", i)
        #This will print once. 
        row['score_player_one'] = None
        row['score_player_two'] = None
     
    df_lh['score_player_one'][i] = row['score_player_one']
    df_lh['score_player_two'][i] = row['score_player_two']
    
df_lh = df_lh.dropna().reset_index(drop=True)

<ipython-input-15-e5e2ec8de999>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['score_player_two'] = 1
<ipython-input-15-e5e2ec8de999>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lh['score_player_one'][i] = row['score_player_one']
<ipython-input-15-e5e2ec8de999>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lh['score_player_two'][i] = row['score_player_two']
<ipython-input-15-e5e2ec8de999>:7: SettingWithCopyWarnin

In [16]:
for attribute in ['template', 'hero_host', 'host_gold', 'guest_gold', 'hero_guest', 'host_color', 'castle_host',
                  'fbhero_host', 'guest_color', 'castle_guest', 'fbhero_guest', 'banned_heroes', 'fb_turn']:
    df_lh[attribute] = [0 for i in range(df_lh.shape[0])]

In [17]:
#Converting the string dictionary to actual dictionary
#This is slow, but we can't get it to work in a better way

for i in range(df_lh.shape[0]):
    if i%1000==0:
        print(i)
        
    dic = try_literal_eval(df_lh['details'][i])    
    
    if 'fbhero_host' not in dic:
        dic['fbhero_host'] = "None"
    if 'fbhero_guest' not in dic:
        dic['fbhero_guest'] = "None"
    
    for attribute in ['template', 'hero_host', 'host_gold', 'guest_gold', 'hero_guest', 'host_color', 'castle_host',
                     'fbhero_host', 'guest_color', 'castle_guest', 'fbhero_guest', 'fb_turn']:
        try:
            df_lh.loc[i, attribute] = dic[attribute]
        except:
            df_lh.loc[i, attribute] = "None"

0
1000
2000
3000
4000
5000


In [18]:
def ending_day_to_number_of_days(text: str):
    '''Takes "116" or and returns 6 or 6. For all errorenuous input, returns None
    Works with both LH and h3GG as I deleted the whitespaces in h3gg version'''
    
    if text is None:
        return None
    elif text == 'w/o' or len(text) != 3:
        return None
    else:
        return int((int(text[0])-1)*28 + (int(text[1])-1)*7 + int(text[2]))

In [19]:
#Removing some rows with non-complete information about game's metadata:
df_lh = df_lh[df_lh['template'] != "--Szablon--"]
df_lh = df_lh[df_lh['castle_host'] != "--Zamek--"]
df_lh = df_lh[df_lh['castle_guest'] != "--Zamek--"]
df_lh = df_lh[df_lh['fb_turn'] != "None"]
dl_lh = df_lh.dropna(subset=['castle_host', 'castle_guest', 'fb_turn']).reset_index(drop=True)

#Removing some useless info: 'details' is already extracted and 'guest gold' is just 'host gold' * -1
df_lh = df_lh.drop(['details', 'guest_gold'], axis=1)

In [20]:
#Changing the naming style so that it's compatible with h3gg:
df_lh = df_lh.rename({'hero_host': 'player_one_hero', 'host_gold': 'bidding_amount',
                     'hero_guest': 'player_two_hero', 'host_color': 'player_one_colour',
                     'castle_host': 'player_one_town', 'guest_color': 'player_two_colour',
                     'castle_guest': 'player_two_town', 'fbhero_host': 'fbhero_player_one',
                     'fbhero_guest': 'fbhero_player_two', 'updated_at': 'created_at', 'fb_turn': 'ending_day'}, axis=1)

In [21]:
#Standardizing all the names to lowercase:
df_lh['player_one_town'] = df_lh['player_one_town'].apply(lambda x: x.lower())
df_lh['player_two_town'] = df_lh['player_two_town'].apply(lambda x: x.lower())

In [22]:
#Converting the ending day into number of days (e.g. 121 into 8)
df_lh['turns_played'] = df_lh['ending_day'].apply(ending_day_to_number_of_days)

In [23]:
#df_lh.to_csv("proper_LH_csv.csv")

# H3gg Dataset collection:

In [24]:
#THIS CELL WILL TAKE A BIT OF TIME! (10 minutes maybe?)

#Run the following cells if and only if your json_data folder is empty for now:
if len(os.listdir('./json_data')) == 0:

    starting_url = 'https://www.h3.gg/api/v1/matches?format=json'
    dicto = requests.get(starting_url).json()
    
    #Read the big API JSON:
    file = json.dumps(dicto, indent = 4)
    with open("json_data/1.json", "w") as f:
        f.write(file)
    
    #If there exists another page of the JSON, read it and proceed until no next page exists:
    i = 1
    while dicto["next"]:
        i += 1
        dicto = requests.get(dicto["next"]).json()
        file = json.dumps(dicto, indent = 4)
        with open(f"json_data/{i}.json", "w") as f:
            f.write(file)

In [25]:
#Read all the JSON files together:
all_results = []
for file_name in os.listdir("json_data"):
    with open(f"json_data/{file_name}", "r") as f:
        file = json.load(f)
        all_results = all_results + file["results"]

In [26]:
def filter_player_info(dic: dict):
    ''''''
    
    if (str(type(dic)) == "<class 'dict'>"):
        return dic['username']
    else:
        return dic
    
    
def filter_game_metadata(dic: dict):
    ''''''
    
    if len(dic) == 0:
        return None

    dic = dic[0]

    if ("source" in dic.keys()):
        return dict((k, dic[k]) for k in ['template', 'player_one_setup', 'player_two_setup', 'description', 
                                          'bidding_amount', 'ending_day', 'started_at', 'created_at'])
    else:
        return dic

In [27]:
df_h3gg = pd.DataFrame(all_results)
df_h3gg = df_h3gg[df_h3gg['finished'] == True]
df_h3gg = df_h3gg[['player_one', 'score_player_one', 'player_two', 'score_player_two', 'winner', 'reportinphase_set']]

df_h3gg['player_one'] = df_h3gg['player_one'].apply(filter_player_info)
df_h3gg['player_two'] = df_h3gg['player_two'].apply(filter_player_info)
df_h3gg['winner'] = df_h3gg['winner'].apply(filter_player_info)
df_h3gg['reportinphase_set'] = df_h3gg['reportinphase_set'].apply(filter_game_metadata)
df_h3gg = df_h3gg.dropna()
df_h3gg = df_h3gg.reset_index(drop = True)

In [28]:
for attribute in ['template', 'player_one_setup', 'player_two_setup', 'description', 'bidding_amount', 
                  'ending_day', 'started_at', 'created_at']:
    df_h3gg[attribute] = [0 for i in range(df_h3gg.shape[0])] #may return bullshit! Actually, works
    

#This is slow as balls, but we can't get it to work in a better way
for i in range(df_h3gg.shape[0]):
    for attribute in ['template', 'player_one_setup', 'player_two_setup', 'description', 
                      'bidding_amount', 'ending_day', 'started_at', 'created_at']:
        df_h3gg.loc[i, attribute] = df_h3gg.loc[i, 'reportinphase_set'][attribute]
        
try:
    df_h3gg = df_h3gg.drop(['reportinphase_set'], axis=1)
except:
    pass

df_h3gg = df_h3gg.dropna()
df_h3gg = df_h3gg.reset_index(drop = True)
df_h3gg['turns_played'] = [None for i in range(df_h3gg.shape[0])]
df_h3gg['player_one_town'] = [None for i in range(df_h3gg.shape[0])]
df_h3gg['player_one_colour'] = [None for i in range(df_h3gg.shape[0])]
df_h3gg['player_two_town'] = [None for i in range(df_h3gg.shape[0])]
df_h3gg['player_two_colour'] = [None for i in range(df_h3gg.shape[0])]
df_h3gg['ending_day'] = df_h3gg['ending_day'].apply(lambda x: x.replace(" ", ""))

In [29]:
df_h3gg['turns_played'] = df_h3gg['ending_day'].apply(ending_day_to_number_of_days)
df_h3gg['player_one_town'] = df_h3gg['player_one_setup'].apply(determining_town)
df_h3gg['player_one_colour'] = df_h3gg['player_one_setup'].apply(determining_colour)
df_h3gg['player_two_town'] = df_h3gg['player_two_setup'].apply(determining_town)
df_h3gg['player_two_colour'] = df_h3gg['player_two_setup'].apply(determining_colour)
df_h3gg = df_h3gg.drop(['started_at', 'description'], axis=1)
df_h3gg['banned_heroes'] = [0] * df_h3gg.shape[0]
df_h3gg['fbhero_player_one'] = [0] * df_h3gg.shape[0]
df_h3gg['fbhero_player_two'] = [0] * df_h3gg.shape[0]
df_h3gg = df_h3gg.dropna()
df_h3gg = df_h3gg.reset_index(drop = True)

In [30]:
def str_to_datetime(text: str):
    '''Remember to re-parse the csv with dates as datetime when loading later on
    The -8 helps combat the .0000000Z ending
    Works with the h3gg dataset, not the LH'''
    try:
        return datetime.strptime(text[:-13], '%Y-%m-%dT%H:%M:%S')
    except:
        return text
    
df_h3gg['created_at'] = df_h3gg['created_at'].apply(str_to_datetime)

#This won't store actual heroes for now, but I want the column names to be equivalent
df_h3gg = df_h3gg.rename({'player_one_setup': 'player_one_hero', 
                          'player_two_setup': 'player_two_hero'}, axis=1)

In [31]:
df_h3gg

,player_one,score_player_one,player_two,score_player_two,winner,template,player_one_hero,player_two_hero,bidding_amount,ending_day,created_at,turns_played,player_one_town,player_one_colour,player_two_town,player_two_colour,banned_heroes,fbhero_player_one,fbhero_player_two
0,Dominacja,1,DaveJustPlayS,0,Dominacja,Jebus Cross,"blue,dungeon,shakti","red,rampart,ivor",2800.0,126,2022-11-19 01:02:34,13.0,dungeon,blue,rampart,red,0,0,0
1,rybas,1,bartosz,0,rybas,6lm10a,"niebieski, inferno, calh","czerwony, cove, anabel",4500.0,147,2022-11-04 16:45:51,28.0,inferno,blue,cove,red,0,0,0
2,rybas,1,kuna999,0,rybas,h3dm3,"niebieski, dungeon, damacon","czerwony, dungeon, sepinroth",-1000.0,114,2022-12-02 19:15:31,4.0,dungeon,blue,dungeon,red,0,0,0
3,rybas,1,amideusz,0,rybas,mt_Andromeda 2.1,"blue, dungeon, damacon","red, dungeon, mutare drake",-1000.0,132,2022-11-17 00:39:21,16.0,dungeon,blue,dungeon,red,0,0,0
4,rybas,1,Koster,0,rybas,mt_Andromeda 2.1,"czerwony, tower, thane","niebieski, tower, cyra",900.0,121,2022-11-29 01:11:17,8.0,tower,red,tower,blue,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,Spinaonic,1,Przemo205,0,Spinaonic,mt_Firewalk,"Blue, Tower, Cyra","Red, Tower, Astral",-6900.0,116,2022-11-11 19:08:04,6.0,tower,blue,tower,red,0,0,0
1682,Spinaonic,1,Dominacja,0,Spinaonic,Jebus Cross,"Red, Cove, Cassiopeia","Blue, Inferno, Calh",3700.0,124,2022-11-26 19:20:15,11.0,cove,red,inferno,blue,0,0,0
1683,Spinaonic,1,DaveJustPlayS,0,Spinaonic,Nostalgia,"Blue. Cove, Anabel","Red, Tower, Solmyr",2200.0,134,2022-11-23 00:21:50,18.0,cove,blue,tower,red,0,0,0
1684,Przemo205,1,Dominacja,0,Przemo205,h3dm3,"Niebieski, Conflux, Aenain","Czerwony, Conflux, Fiur",-1000.0,131,2022-12-17 00:31:59,15.0,conflux,blue,conflux,red,0,0,0


In [32]:
#df_h3gg.to_csv("proper_csv.csv")

### Merging the two datasets:

In [33]:
df_lh = df_lh[df_h3gg.columns]
df_combined = pd.concat([df_h3gg, df_lh]).sort_values(by=['created_at'])
df_combined = df_combined[df_combined['player_one_town'] != '--Zamek--'].reset_index(drop=True)
df_combined = df_combined.drop(['banned_heroes', 'fbhero_player_one', 'fbhero_player_two'], axis=1)

In [34]:
#Now, we need to convert the fake 2-1 scores into 1-0 for consistency
for i in range(df_combined.shape[0]):
    #In case of a 2-1, reduce both by 1
    if (df_combined.loc[i, 'score_player_one'] + df_combined.loc[i, 'score_player_two'] == 3):
        df_combined.loc[i, 'score_player_one'] -= 1
        df_combined.loc[i, 'score_player_two'] -= 1
    #In case of a 2-0, reduce the 2 by 1
    elif (df_combined.loc[i, 'score_player_one'] == 2):
        df_combined.loc[i, 'score_player_one'] -= 1
    #In case of a 0-2, reduce the 2 by 1
    elif (df_combined.loc[i, 'score_player_two'] == 2):
        df_combined.loc[i, 'score_player_two'] -= 1
    #else: pass

### Now, the hard part. We need to extract the hero names from the h3gg's user input for setups:

### beware:
Adela-Adelaide, Astra-Astral, Axsis-Xsi!

Especially Adela-Adelaide. Since they come from the same town, any occurence of Adela will also match with Adelaide, but not the other way around. This is why despite the stadnard ordering, Adelaide HAS TO come before Adela in the order, so that we first try matching on her name

In [35]:
def lowercasesize_array(array: list):
    return [word.lower() for word in array]

In [36]:
def extracting_heroes(row: pd.Series, heroes_dict: dict, simplification_dict: dict):
    '''Replace the row of your dataset with the row returned by this'''
    
    #Returning lists of heroes potentially chosen by the player (based on their starting town):
    player_one_possible_heroes = lowercasesize_array(heroes_dict[row['player_one_town'].lower()]) + ['']
    player_two_possible_heroes = lowercasesize_array(heroes_dict[row['player_two_town'].lower()]) + ['']
    
    #PRODUCTION CODE: LEAVE THE + '' THERE. IT WILL GET RID OF ALL NON-STANDARD MISSPELLS
    #IF NOT PRODUCTION CODE, REMOVE THE + '' THERE!
    
    #player_one_hero_as_inputted = row['player_one_hero'].lower().replace('/', ' ').replace('>', ' ')
    
    #Fixing player one's hero:
    for hero in player_one_possible_heroes:
        
        #If the given real hero occurs in whatever the user inputted
        if hero in row['player_one_hero'].lower():
            row['player_one_hero'] = hero
            #If besides being "real" it is also in the simplification dict, then it was a misspell:
            if hero in simplification_dict:
                row['player_one_hero'] = simplification_dict[hero]
            break
    if hero == '':
        row['player_one_hero'] = 'None'
    
    #Fixing player one's hero:
    for hero in player_two_possible_heroes:
        
        #If the given real hero occurs in whatever the user inputted
        if hero in row['player_two_hero'].lower():
            row['player_two_hero'] = hero
            #If besides being "real" it is also in the simplification dict, then it was a misspell:
            if hero in simplification_dict:
                row['player_two_hero'] = simplification_dict[hero]
            break
    if hero == '':
        row['player_two_hero'] = 'None'
            
    return row

In [37]:
for i in range(df_combined.shape[0]):
    if i%500==0:
        print(i)
    df_combined.loc[i] = extracting_heroes(df_combined.loc[i], heroes, simplification_dict)

0


<ipython-input-36-a7095b09b2f5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['player_one_hero'] = hero
<ipython-input-36-a7095b09b2f5>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['player_two_hero'] = hero


500


<ipython-input-36-a7095b09b2f5>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['player_one_hero'] = 'None'
<ipython-input-36-a7095b09b2f5>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['player_two_hero'] = 'None'


1000
1500


<ipython-input-36-a7095b09b2f5>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['player_two_hero'] = simplification_dict[hero]
<ipython-input-36-a7095b09b2f5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['player_one_hero'] = simplification_dict[hero]


2000
2500
3000
3500
4000


In [38]:
#Removing the non-standard misspells and rows with no information:
df_combined = df_combined[df_combined['player_one_hero'] != 'None']
df_combined = df_combined[df_combined['player_two_hero'] != 'None']
#Colour = white only means the game resulted in a technical loss and no actual game was played:
df_combined = df_combined[df_combined['player_one_colour'] != 'white']
df_combined = df_combined[df_combined['player_two_colour'] != 'white'].reset_index(drop=True)

In [39]:
df_combined['template'].unique()

array(['nostalgia', 'h3dm1', 'mt_fw', '6lm10a', 'mt_andromeda',
       'mininosta', 'mt_mp', 'mt_antares', 'w/o', 'rally', 'h3dm3.2',
       'spider', 'h3dm3', 'Nostalgia', 'mt_Firewalk', 'Jebus Cross',
       'mt_Andromeda 2.1', 'mt_JebusKing', 'Duel', '8mm6a', 'mt_wrzosy',
       '', 'Balance', '8xm12a', 'nostalgiaTP', 'mt_andromeda2.1',
       'mini-nosta', '2sm4d(3)', 'mt_Nebula', 'mt_mp 1.04', 'Sapphire',
       'kubaryt', 'mt_Andromeda 2.2'], dtype=object)

We will need to keep on fixing the misspells until the unique names of heroes only are the ones that actually exist in the game. I have dont this manually and trust me - it took a LOT of time to get it right!

In [40]:
all_heroes_entries = np.concatenate([df_combined['player_one_hero'].unique(), df_combined['player_two_hero'].unique()])
all_heroes_entries

array(['solmyr', 'gundula', 'giselle', 'octavia', 'leena', 'eovacius',
       'thant', 'luna', 'aine', 'alkin', 'coronius', 'anabel', 'grindan',
       'valeska', 'daremyth', 'wystan', 'jeremy', 'shiva', 'ivor',
       'jenova', 'nymus', 'illor', 'shakti', 'tyraxor', 'monere',
       'astral', 'lorelei', 'drakon', 'aenain', 'calh', 'bron', 'xyron',
       'marius', 'tazar', 'damacon', 'brissa', 'sorsha', 'andra',
       'lord haart', 'beatrice', 'aeris', 'adelaide', 'mutare drake',
       'caitlin', 'ajit', 'gretchin', 'tyris', 'sandro', 'mephala',
       'elleshar', 'cyra', 'septienna', 'adela', 'neela', 'alamar',
       'crag hack', 'jabarkas', 'dace', 'gelare', 'isra', 'axsis',
       'mirlanda', 'sephinroth', 'pasis', 'deemer', 'boragus', 'ignatius',
       'corkes', 'erdamon', 'ciele', 'clavius', 'terek', 'cuthbert',
       'ayden', 'cassiopeia', 'tamika', 'vidomina', 'derek', 'nagash',
       'adrienne', 'andal', 'charna', 'iona', 'rosic', 'malcom',
       'aislinn', 'josephine',

Here, we simply fix the templates' names by merging and bin them according to their "types".

The merging is done to equalize different versions of the same template

For example, h3dm1 later got a tiny fix (really tiny) and became h3dm2. Another tiny fix - h3dm3.2. Another one - h3dm3

The binning is done so that we compare uniformly over similar templates. For example, Spider and 6lm10a are roughly identical in gameplay

### The functions to perform this are present in the library - they're super simple, yet lengthy

In [41]:
df_combined['template'] = df_combined['template'].apply(merging_templates)
df_combined['template_type'] = [0]*df_combined.shape[0]
for i in range(df_combined.shape[0]):
    df_combined.loc[i, 'template_type'] = binning_templates(df_combined['template'][i])

In [42]:
df_combined

,player_one,score_player_one,player_two,score_player_two,winner,template,player_one_hero,player_two_hero,bidding_amount,ending_day,created_at,turns_played,player_one_town,player_one_colour,player_two_town,player_two_colour,template_type
0,michail117,0,[SDK] Ravciozo,1,[SDK] Ravciozo,Nostalgia/TP,solmyr,gunnar,2200,134,2021-04-29 16:54:05,18.0,tower,red,dungeon,blue,XL+U
1,HawkWRz,0,Vertr0X,1,Vertr0X,h3dm1/3,gundula,crag hack,900,124,2021-05-19 08:21:53,11.0,stronghold,red,stronghold,blue,Mirror
2,Bartt666,0,BOARtek,1,BOARtek,h3dm1/3,giselle,aeris,900,143,2021-05-20 14:49:18,24.0,rampart,red,rampart,blue,Mirror
3,BOARtek,0,GALAXER00,1,GALAXER00,mt_Firewalk,octavia,xyron,-5500,136,2021-05-21 14:44:08,20.0,inferno,red,inferno,blue,Mirror
4,Fafhrd,1,GALAXER00,0,Fafhrd,h3dm1/3,leena,jeremy,-900,122,2021-05-21 20:33:37,9.0,cove,red,cove,blue,Mirror
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Odelle,1,bartosz,0,Odelle,mt_Andromeda,rissa,cyra,-1100.0,124,2023-04-02 22:15:09,11.0,tower,red,tower,blue,Mirror
4022,Scyzoryk,0,mirek_swirek,1,mirek_swirek,Jebus Cross,shakti,solmyr,4400.0,115,2023-04-02 22:31:17,5.0,dungeon,blue,tower,red,Jebus
4023,neSpeshaPlay,1,eska,0,neSpeshaPlay,Jebus Cross,valeska,shakti,-2000.0,122,2023-04-04 12:58:38,9.0,castle,red,dungeon,blue,Jebus
4024,Akabes,1,juro,0,Akabes,mt_Firewalk,mephala,jenova,-5500.0,115,2023-04-04 18:27:12,5.0,rampart,blue,rampart,red,Mirror


In [43]:
df_combined.to_csv("combined_processed.csv", index=False)

## Transposing the dataframe into a format easily used by the dashboard

In [45]:
df = pd.read_csv("combined_processed.csv")

In [46]:
df2 = pd.DataFrame(columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", "turns", "template"])

for row in df.iterrows():
    row = row[1]
    if row[1] > row[3]: result = 1
    elif row[1] < row[3]: result = 0
    else: result = 0.5
    df_temp = pd.DataFrame([[result, row[12], row[6], row[13], row[8] , row[14], row[7], row[11], row[5], row[16]]], 
                          columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", 
                                     "turns", "template", "template_type"])
    df2 = pd.concat([df2, df_temp])

    df_temp = pd.DataFrame([[1-result, row[14], row[7], {"red": "blue", "blue": "red", "white": "white"}[row[13]], 
                             -row[8], row[12], row[6],row[11], row[5], row[16]]], 
                           columns = ["result", "town", "hero", "color", "bidding", "opponent_town", "opponent_hero", 
                                      "turns", "template", "template_type"])
    df2 = pd.concat([df2, df_temp])

df2.to_csv("set2.csv", index = False)